In [4]:
import psycopg2
import json
import numpy as np
from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer

# Database configuration
db_config = {
    'dbname': 'wb_s2_embeddings',
    'user': 's2',
    'password': 'wb@s2',
    'host': 'localhost',
    'port': 5432
}

model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')


query = "Remote sensing for fertilizer management"

# Load the projects
with open("digital_agriculture_projects.json", "r") as f:
    projects = json.load(f)

# Function to perform the search
def search(query, model, db_config):
    # Encode the query
    query_embedding = model.encode([query])[0]
    query_embedding = [float(value) for value in query_embedding]

    
    # Connect to the database
    with psycopg2.connect(**db_config) as conn:
        c = conn.cursor()

        # Search for the top 5 projects by cosine similarity
        c.execute("""
            SELECT project_id, chunk, embedding <-> %s::VECTOR AS distance
            FROM embeddings
            ORDER BY distance ASC
            LIMIT 5;
        """, (list(query_embedding),))
        
        results = c.fetchall()
    
    # Print the results
    for project_id, chunk, distance in results:
        print(f"Project ID: {project_id}")
        print(f"Project title: {projects[project_id]['title']}")
        print(f"Project abstract: {projects[project_id]['abstract']}")
        print(f"Relevant snippet: {chunk}")
        print(f"Distance: {distance}")
        print("\n")

# Initialize the model
model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')

# Define the database configuration
db_config = {
    'dbname': 'wb_s2_embeddings',
    'user': 's2',
    'password': 'wb@s2',
    'host': 'localhost',
    'port': 5432
}

# Define the query
query = "Use of satellite imagery remote sensing for fertilizer management in Morocco"

# Perform the search
search(query, model, db_config)
